In [1]:
!pip install xgboost imbalanced-learn shap

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score, classification_report

from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

import joblib

In [3]:
from google.colab import files
uploaded = files.upload()

Saving cardio_train.csv to cardio_train (1).csv


In [4]:
df = pd.read_csv("cardio_train.csv", sep=';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [5]:
# Remove ID column
df.drop("id", axis=1, inplace=True)

# Convert age from days to years
df['age'] = df['age'] / 365

# Remove impossible BP values
df = df[(df['ap_hi'] < 250) & (df['ap_lo'] < 200)]
df = df[(df['ap_hi'] > 50) & (df['ap_lo'] > 30)]

df.shape

(68775, 12)

In [6]:
df['bmi'] = df['weight'] / ((df['height']/100)**2)

In [30]:
df.rename(columns={'cardio': 'target_cardio'}, inplace=True)

In [31]:
np.random.seed(42)

df['avg_daily_steps'] = np.random.normal(6000, 2000, len(df))
df['active_minutes'] = np.random.normal(45, 20, len(df))
df['resting_heart_rate'] = np.random.normal(72, 8, len(df))
df['calories_burned'] = np.random.normal(2200, 400, len(df))
df['sleep_hours'] = np.random.normal(7, 1.2, len(df))

In [32]:
df['aqi_exposure'] = np.random.normal(120, 40, len(df))
df['temperature'] = np.random.normal(35, 6, len(df))
df['humidity'] = np.random.normal(60, 15, len(df))

In [10]:
df['target_respiratory'] = (
    (df['smoking'] == 1) &
    (df['aqi_exposure'] > 130)
).astype(int)

In [11]:
df['target_heat'] = (
    (df['age'] > 50) &
    (df['bmi'] > 30) &
    (df['heat_exposure'] > 40)
).astype(int)

In [33]:
features = [
    'age',
    'gender',
    'bmi',
    'avg_daily_steps',
    'active_minutes',
    'resting_heart_rate',
    'calories_burned',
    'sleep_hours',
    'aqi_exposure',
    'temperature',
    'humidity'
]

X = df[features]

y = df[['target_cardio', 'target_respiratory', 'target_heat']]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [35]:
base_model = XGBClassifier(
    n_estimators=400,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss'
)

multi_model = MultiOutputClassifier(base_model)

multi_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.8, device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='logloss',
                                              feature_types=None,
                                              feature_weights=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=0.05, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=5,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=400, n_jobs=None,
                                              num_parallel_tree=None, ...))

In [36]:
y_pred_proba = multi_model.predict_proba(X_test)

for i, col in enumerate(y.columns):
    auc = roc_auc_score(y_test[col], y_pred_proba[i][:,1])
    print(f"{col} AUC: {auc:.4f}")

target_cardio AUC: 0.6706
target_respiratory AUC: 0.9196
target_heat AUC: 1.0000


In [38]:
def predict_health_risk(model, user_data):

    probs = model.predict_proba(user_data)

    cardio = probs[0][0][1]
    respiratory = probs[1][0][1]
    heat = probs[2][0][1]

    overall = (cardio + respiratory + heat) / 3

    # Risk classification function
    def classify_risk(score):
        if score < 0.30:
            return "🟢 LOW RISK"
        elif score < 0.60:
            return "🟡 MODERATE RISK"
        else:
            return "🔴 HIGH RISK"

    print("\n" + "="*60)
    print("🏥            PERSONAL HEALTH RISK REPORT")
    print("="*60)

    print(f"\n❤️ Cardiovascular Risk  : {cardio:.2f}  →  {classify_risk(cardio)}")
    print(f"🌫️ Respiratory Risk     : {respiratory:.2f}  →  {classify_risk(respiratory)}")
    print(f"🌡️ Heat Vulnerability   : {heat:.2f}  →  {classify_risk(heat)}")

    print("\n" + "-"*60)
    print(f"📊 OVERALL HEALTH SCORE : {overall:.2f}  →  {classify_risk(overall)}")
    print("-"*60)

    # Overall recommendation
    if overall < 0.30:
        print("💚 Excellent condition! Maintain your healthy lifestyle.")
    elif overall < 0.60:
        print("⚠️ Moderate risk detected. Consider improving daily habits.")
    else:
        print("🚨 High risk detected. Medical consultation is recommended.")

    print("="*60)

In [39]:
sample_user = pd.DataFrame([{
    'age': 45,
    'gender': 1,
    'bmi': 31,
    'avg_daily_steps': 4000,
    'active_minutes': 20,
    'resting_heart_rate': 82,
    'calories_burned': 1800,
    'sleep_hours': 5,
    'aqi_exposure': 190,
    'temperature': 41,
    'humidity': 75
}])

predict_health_risk(multi_model, sample_user)


🏥            PERSONAL HEALTH RISK REPORT

❤️ Cardiovascular Risk  : 0.38  →  🟡 MODERATE RISK
🌫️ Respiratory Risk     : 0.00  →  🟢 LOW RISK
🌡️ Heat Vulnerability   : 0.00  →  🟢 LOW RISK

------------------------------------------------------------
📊 OVERALL HEALTH SCORE : 0.13  →  🟢 LOW RISK
------------------------------------------------------------
💚 Excellent condition! Maintain your healthy lifestyle.


In [40]:
import joblib

# Save trained model
joblib.dump(multi_model, "health_risk_model.pkl")

# Save feature order (VERY IMPORTANT for backend)
joblib.dump(features, "model_features.pkl")

print("✅ Model and feature list saved successfully!")

✅ Model and feature list saved successfully!
